In [1]:
from pvtpy.pvt import PVT
import pvtpy.black_oil as bk
from pvtpy.fluids import Gas, InitialConditions
from pvtpy.pvt import PVT, Chromatography
import numpy as np
import pandas as pd
import seaborn as sns

# Create a chromatography composition

In [2]:

components=['carbon-dioxide','nitrogen','methane','ethane','propane','isobutane','n-butane']
mole_fraction = [0.02,0.01,0.85,0.04,0.03,0.03,0.02]

chr = Chromatography(
    components = components,
    mole_fraction = mole_fraction
)

print(chr.df())

    id            name formula  mole_fraction       mw     ppc     tpc
52  53  carbon-dioxide     CO2           0.02  44.0100  1071.0   87.91
59  60        nitrogen      N2           0.01  28.0134   493.1 -232.51
0    1         methane     CH4           0.85  16.0430   666.4 -116.67
1    2          ethane    C2H6           0.04  30.0700   706.5   89.92
2    3         propane    C3H8           0.03  44.0970   616.0  206.06
3    4       isobutane   C4H10           0.03  58.1230   527.9  274.46
4    5        n-butane   C4H10           0.02  58.1230   550.6  305.62


## Estimate some properties

### Apparent Molecular Weigt

The apparent Molecular Weight (ma) is calculated by summing the product of molar fraction and molecular weight of each component in the chromatography

In [3]:
chr.mwa()

20.228744

### Gas specific gravity

The Gas specific gravity is calculated by diving the **ma** by the specific gravity of the *air*

In [4]:
chr.gas_sg()

0.6985063535911602

### Pseudo critical properties

The Pseudo critical properties are calulated by summing the product of mole fraction and critical properties (pressure and temperature). By default it corrects the properties by Non-hydrocarbon components with the **wichert-aziz** correlation. 

In [5]:
chr.get_pseudo_critical_properties()

CriticalProperties(ppc=695.6353554009818, tpc=-78.93109372795836)

In [6]:
chr.get_pseudo_critical_properties(correct=False)

CriticalProperties(ppc=666.3799999999999, tpc=-75.61160000000001)

In [7]:
chr.get_pseudo_critical_properties(correct_method='carr_kobayashi_burrows')

CriticalProperties(ppc=673.4799999999998, tpc=-79.7116)

### Get the compressibility factor of gas

Estimate the compressibility factor by estimating the critical properties and applying the default correlation method **papay**

In [8]:
chr.get_z(pressure=3000, temperature=180)

,z
pressure,
3000,0.873299


In [9]:
p_range = np.linspace(1000,5000,10)
chr.get_z(pressure=p_range, temperature=180)

,z
pressure,
1000.000000,0.940261
1444.444444,0.919329
1888.888889,0.901855
2333.333333,0.887839
2777.777778,0.877281
3222.222222,0.870181
3666.666667,0.866539
4111.111111,0.866355
4555.555556,0.869629


### Get the gas density in lb/ft3

Estimate the gas density by estimating the **ma**, the **z** factor and finnaly applying the gas equation of state for **real gases**

In [10]:
chr.get_rhog(pressure=3000,temperature=180)

,rhog
pressure,
3000,10.11923


In [11]:
chr.get_rhog(pressure=3000,temperature=180, rhog_method='ideal_gas')

,rhog
pressure,
3000,8.837114


In [12]:
chr.get_rhog(pressure=np.linspace(1000,5000,10),temperature=180,rhog_method='real_gas')

,rhog
pressure,
1000.000000,3.132860
1444.444444,4.628275
1888.888889,6.169627
2333.333333,7.741617
2777.777778,9.327126
3222.222222,10.907744
3666.666667,12.464429
4111.111111,13.978238
4555.555556,15.431087


### Estimate the Specific volume of Gas. 

Get the specific volume by estimate the inverse of the density

In [13]:
chr.get_sv(pressure=3000,temperature=180, rhog_method='ideal_gas')

pressure
3000    0.113159
Name: sv, dtype: float64

In [14]:
chr.get_sv(pressure=3000,temperature=180, rhog_method='real_gas')

pressure
3000    0.098822
Name: sv, dtype: float64

# Create the Gas object

In [15]:
init = InitialConditions(pressure=3600, temperature=160)

In [16]:
gas1 = Gas(
    initial_conditions=init,sg=0.6,
    chromatography=chr
)


In [17]:
gas1.pseudo_critical_properties()

CriticalProperties(ppc=695.6353554009818, tpc=-78.93109372795836)

In [18]:
gas1.pvt_from_correlations()
gas1.pvt.df()

,z,rhog,bg,mug,cg
pressure,,,,,
20.000000,0.998462,0.060908,0.155690,0.012481,0.050000
282.105263,0.979014,0.876193,0.010823,0.012693,0.003545
544.210526,0.960894,1.722143,0.005506,0.013021,0.001838
806.315789,0.944100,2.596956,0.003652,0.013438,0.001240
1068.421053,0.928634,3.498446,0.002711,0.013938,0.000936
1330.526316,0.914496,4.424043,0.002143,0.014522,0.000752
1592.631579,0.901684,5.370792,0.001766,0.015193,0.000628
1854.736842,0.890200,6.335374,0.001497,0.015955,0.000539
2116.842105,0.880043,7.314121,0.001297,0.016811,0.000472
